# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [277]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [278]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-22-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [279]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [280]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [281]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) 
    session = cluster.connect()
except Exception as e:
    print(e)

    
# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [282]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS dend_proj2
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [283]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('dend_proj2')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### Create table to run the following queries based on the query 1

In [284]:
#Keys Available:
#artist text[0], firstName text[1], gender text[2], itemInSession text[3], lastName text[4], 
#length text[5], level text[6], location text[7], sessionId text[8], song text[9], userId text[10] 

In [285]:
#QUERY 1 - keys needed:
#SELECT:     artist text[0], song text[9], length text[5]
#WHERE:  itemInSession text[3] = 4 , sessionId text[8] = 338,

In [286]:
query = "CREATE TABLE IF NOT EXISTS session338_library "
query = query + "(artist text, song text, length text, sessionId text, itemInSession text,  \
                  PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [287]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        
        query = "INSERT INTO session338_library (artist, song, length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (line[0], line[9], line[5], line[8], line[3]))

In [288]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM session338_library \
         WHERE sessionId = '338' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)
        
#    print (row.itemInSession, row.sessionId, row.artist, row.song, row.length)

Row(sessionid='338', iteminsession='1', artist='Pixies', length='89.36444', song='Build High')
Row(sessionid='338', iteminsession='2', artist='The Roots / Jack Davey', length='155.95057', song='Atonement')
Row(sessionid='338', iteminsession='3', artist='Mike And The Mechanics', length='275.12118', song='A Beggar On A Beach Of Gold')
Row(sessionid='338', iteminsession='4', artist='Faithless', length='495.3073', song='Music Matters (Mark Knight Dub)')


In [289]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "SELECT artist, song, length FROM session338_library \
         WHERE sessionId = '338' AND itemInSession = '4' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [290]:
# Available keys:
#artist text[0], firstName text[1], gender text[2], itemInSession text[3], lastName text[4], 
#length text[5], level text[6], location text[7], sessionId text[8], song text[9], userId text[10] 

In [291]:
## Query 2: keys needed:
#SELECT:    artist text[0], song text[9]
#WHERE:     sessionId text[8] = 182, userId text[10] = 10
#ORDER BY:  itemInSession text[3], firstName text[1], lastName text[4]

In [292]:
query = "CREATE TABLE IF NOT EXISTS session182_songlog "
query = query + "(artist text, song text, itemInSession text, firstName text, \
                  lastName text, sessionId text, userId text, PRIMARY KEY ((sessionId, userId), itemInSession, firstName, lastName))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [293]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        
        query = "INSERT INTO session182_songlog (artist, song, itemInSession, firstName, \
                                            lastName, sessionId, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (line[0], line[9], line[3], line[1], line[4], line[8], line[10]))

In [294]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT sessionId, userid, firstName, lastName, artist, song, itemInSession  \
         FROM session182_songlog \
         WHERE sessionId = '269' AND userid='44' ";


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)
#    print(row.itemInSession, row.firstName, row.lastName)

Row(sessionid='269', userid='44', firstname='Aleena', lastname='Kirby', artist='Feist', song='My Moon My Man', iteminsession='10')
Row(sessionid='269', userid='44', firstname='Aleena', lastname='Kirby', artist='Flobots', song='Handlebars (UK Radio Edit)', iteminsession='11')
Row(sessionid='269', userid='44', firstname='Aleena', lastname='Kirby', artist='Dwight Yoakam', song="You're The One", iteminsession='12')
Row(sessionid='269', userid='44', firstname='Aleena', lastname='Kirby', artist='The Pussycat Dolls', song='When I Grow Up', iteminsession='13')
Row(sessionid='269', userid='44', firstname='Aleena', lastname='Kirby', artist='The Prodigy', song='Omen', iteminsession='14')
Row(sessionid='269', userid='44', firstname='Aleena', lastname='Kirby', artist='Caifanes', song='Cuentame Tu Vida', iteminsession='15')
Row(sessionid='269', userid='44', firstname='Aleena', lastname='Kirby', artist='Jimmy Buffett', song='He Went To Paris', iteminsession='19')
Row(sessionid='269', userid='44', fir

In [295]:
# Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) 
#           for userid = 10, sessionid = 182

query = "SELECT artist, song  \
         FROM session182_songlog \
         WHERE sessionId = '182' AND userid = '10'  ORDER BY itemInSession, firstName, lastName";

#sessionId, userId, itemInSession, firstName, lastName


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)
#    print(row.artist, row.song, row.itemInSession, row.firstName, row.lastName)

Row(artist='Down To The Bone', song="Keep On Keepin' On")
Row(artist='Three Drives', song='Greece 2000')
Row(artist='Sebastien Tellier', song='Kilometer')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [296]:
# Available keys:
#artist text[0], firstName text[1], gender text[2], itemInSession text[3], lastName text[4], 
#length text[5], level text[6], location text[7], sessionId text[8], song text[9], userId text[10] 

In [297]:
# Query 3: Keys needed:
#SELECT:     firstName text[1], lastName text[4]
#SORTED BY: none
#WHERE:     song text[9] = 'All Hands Against His Own'

In [298]:
query = "CREATE TABLE IF NOT EXISTS song_all_hands_events "
query = query + "(firstName text, lastName text, song text, PRIMARY KEY ((song), firstName, lastName))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [299]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        
        query = "INSERT INTO song_all_hands_events (firstName, lastName, song)"
        query = query + " VALUES (%s, %s, %s)"

        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[1], line[4], line[9]))

In [301]:
## Add in the SELECT statement to verify the data was entered into the table
query = " SELECT * FROM song_all_hands_events \
          WHERE song='All Hands Against His Own' "

try:
    rows = session.execute(query)

except Exception as e:
    print(e)
    
for row in rows:
    print(row)
        
#    print (row.itemInSession, row.sessionId, row.artist, row.song, row.length)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')


In [302]:
## Add in the SELECT statement to verify the data was entered into the table
query = " SELECT firstName, lastName FROM song_all_hands_events  \
          WHERE song = 'All Hands Against His Own'"

#SELECT firstName, lastName WHERE song = 'All Hands Against His Own'
try:
    rows = session.execute(query)

except Exception as e:
    print(e)
    
for row in rows:
    print(row)
        
#    print (row.itemInSession, row.sessionId, row.artist, row.song, row.length)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Sara', lastname='Johnson')
Row(firstname='Tegan', lastname='Levine')


As we can see there is no result for song that equals 'All Hands Against His Own' in this library table.

### Drop the tables before closing out the sessions

In [303]:
## TO-DO: Drop the table before closing out the sessions

In [304]:
query = "drop table session338_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table session182_songlog"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_all_hands_events"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [276]:
session.shutdown()
cluster.shutdown()